Model for mean

In [154]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go
import matplotlib.pyplot as plt
import seaborn as sns
import networkx as nx
import scipy as sc
import keras

In [155]:
df_mean = pd.read_csv('data/df_mean.csv', index_col=0)
df_mean.head()

,Rbin,mean,v,npt
0,67600,-245.015625,0.7,2.0
1,67100,49.808594,0.7,2.0
2,66900,186.148438,0.7,1.0
3,67400,413.269531,0.7,2.0
4,67000,278.406250,0.7,1.0


In [156]:
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.neural_network import MLPRegressor
from sklearn.cluster import KMeans
from sklearn.preprocessing import PolynomialFeatures
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

import tensorflow as tf
import statsmodels.api as sm

from keras import Sequential

from xgboost import XGBRegressor

In [157]:
data_mean = df_mean.dropna()
data_mean = data_mean.astype(float)
data_mean.head()

,Rbin,mean,v,npt
0,67600.0,-245.015625,0.7,2.0
1,67100.0,49.808594,0.7,2.0
2,66900.0,186.148438,0.7,1.0
3,67400.0,413.269531,0.7,2.0
4,67000.0,278.406250,0.7,1.0


In [158]:
# for val in data_mean['v'].unique():
#     RB = (77.7*np.square(val) - 74.4*val + 112)*1000
#     data_mean.loc[data_mean['v']==val, 'RB'] = RB
# data_mean.head()

In [159]:
# data_mean = data_mean.astype(float)
# X_train, X_test, Y_train, Y_test = train_test_split(data_mean[['Rbin', 'v', 'npt']],
#                                                     data_mean['mean'], test_size=0.2, random_state=42)

In [180]:
vval = 1.6
# Prepare data of R_delta_mean for training and testing
# data_mean = data_mean.loc[data_mean['v']>=1]
X_train = data_mean.loc[(data_mean['v']<vval), ['Rbin', 'v', 'npt']]
X_test = data_mean.loc[(data_mean['v']>=vval), ['Rbin', 'v', 'npt']]
Y_train = data_mean.loc[(data_mean['v']<vval), 'mean']
Y_test = data_mean.loc[(data_mean['v']>=vval), 'mean']

In [181]:
x_train, x_val, y_train, y_val = train_test_split(X_train, Y_train, test_size=0.5, random_state=50)

In [182]:
from sklearn.preprocessing import PolynomialFeatures, PowerTransformer
from sklearn.preprocessing import StandardScaler, RobustScaler, MinMaxScaler, MaxAbsScaler

scaler = StandardScaler()
pf = PolynomialFeatures(degree=3, include_bias=False, interaction_only=False)
pf2 = PolynomialFeatures(degree=2)

x_train_scaled = scaler.fit_transform(x_train)
x_val_scaled = scaler.transform(x_val)
X_test_scaled = scaler.transform(X_test)

x_train_pf = pf.fit_transform(x_train_scaled)
x_val_pf = pf.transform(x_val_scaled)
X_test_pf = pf.transform(X_test_scaled)

In [183]:
pf3 = Pipeline(steps=[
    ("scaler", StandardScaler()),             # Scale the transformed features
    ("poly3", PolynomialFeatures(degree=3, include_bias=False, interaction_only=False)),  # Apply polynomial transformation
])
pf2 = Pipeline(steps=[
    ("scaler", StandardScaler()),             # Scale the transformed features
    ("poly2", PolynomialFeatures(degree=2, include_bias=False, interaction_only=False)),  # Apply polynomial transformation
])
pf1 = Pipeline(steps=[
    ("scaler", StandardScaler()),             # Scale the transformed features
    ("poly1", PolynomialFeatures(degree=1, include_bias=False, interaction_only=False)),  # Apply polynomial transformation
])


ct = ColumnTransformer(transformers=[
    ("num", pf3, ["Rbin", "v", "npt"]),
#     ("v", pf1, ["v"]),
#     ("npt", pf1, ["npt"]),
])

In [184]:
x_train_pf = ct.fit_transform(x_train)
x_val_pf = ct.transform(x_val)
X_test_pf = ct.transform(X_test)

In [185]:
# ## Model for R_delta_mean
# model = XGBRegressor(n_estimators=1000, max_depth=5, eta=0.001, subsample=0.5,
#                      eval_metric="logloss", early_stopping_rounds=20)
# model.fit(x_train_pf, y_train, eval_set=[(x_val_pf, y_val)], verbose=True)
# y_val_pred = model.predict(x_val_pf)

In [186]:
# # Weighted Linear Regression
# model = LinearRegression()

# model.fit(x_train_pf, y_train)
# predictions = model.predict(x_train_pf)
# mean_err = abs(y_train - predictions.mean())

# weights = np.log(x_train['npt'])
# # display(err_var)

# model = LinearRegression().fit(x_train_pf, y_train, sample_weight=weights)
# weights

In [187]:
# model = LinearRegression()
# model.fit(x_train_pf, y_train)

In [188]:
model0 = LinearRegression()
model0.fit(x_train_pf, y_train)
y_train_pred = model0.predict(x_train_pf)
model0.score(x_train_pf, y_train)

0.6985555665110978

In [189]:
residuals = y_train - y_train_pred
residual_variances = np.var(residuals)
weights = 1

In [190]:
model = sm.WLS(y_train, x_train_pf, weights=weights)
model = model.fit()
model.rsquared

0.73526094530015

In [191]:
# Model validation
y_val_pred = model.predict(x_val_pf)

In [192]:
# # Model scores
# mse = mean_squared_error(y_val, y_val_pred)
# r_score = model.score(x_val_pf, y_val)
# print(f"mse: {mse}")
# print(f"r2: {r_score}")

In [193]:
# # Test score
y_test_pred = model.predict(X_test_pf)
# print(f"r2: {model.score(X_test_pf, Y_test)}")

In [194]:
# fig = go.Figure()
# fig.add_trace(go.Scatter(y = Y_test,
#                             x = X_test['Rbin'],
#                             marker=dict(color='blue'),
#                             mode='markers',
#                             name=f'{X_test['v'].mean()}V'
#                             ))
# fig.add_trace(go.Scatter(y = y_test_pred,
#                             x = X_test['Rbin'],
#                             line=dict(color='red'),
#                             mode='markers',
#                             name='Prediction'
#                             ))
# fig.update_layout(
# title="\u0394R\u03C3 vs Rbin",
# xaxis_title="Rbin",
# yaxis_title="\u0394R\u03C3"
# )

In [195]:
plot_df = pd.DataFrame()
# test data
plot_df['v'] = X_test['v']
plot_df['Rbin'] = X_test['Rbin']
plot_df['Y_test'] = Y_test
plot_df['y_test_pred'] = y_test_pred
plot_df = plot_df.sort_values('v')
plot_df.head()

,v,Rbin,Y_test,y_test_pred
8534,1.6,111500.0,9161.953125,15579.949597
9021,1.6,188700.0,59.613517,-184.557714
9022,1.6,197700.0,-1031.243490,-2054.753630
9023,1.6,134800.0,6439.734375,5765.436515
9024,1.6,138000.0,7019.796875,4787.175111


In [196]:
colours = ['#636EFA', '#EF553B', '#00CC96', '#AB63FA', '#FFA15A', '#19D3F3', '#FF6692', '#B6E880', '#FF97FF', '#FECB52',
           '#636EFA', '#EF553B', '#00CC96', '#AB63FA', '#FFA15A', '#19D3F3', '#FF6692', '#B6E880', '#FF97FF', '#FECB52'
           ]
fig = go.Figure()

for val, col in zip(plot_df['v'].unique(), colours):
    data = plot_df.loc[plot_df['v']==val]
    fig.add_trace(go.Scatter(y = data['Y_test'],
                             x = data['Rbin'],
                             marker=dict(color=col),
                             mode='markers',
                             name=f'{val}V'
                             ))
    fig.add_trace(go.Scatter(y = data['y_test_pred'],
                             x = data['Rbin'],
                             line=dict(color='red'),
                             mode='markers',
                             name=f'{val}V'
                             ))
fig.update_layout(
    title="\u0394R\u03C3 vs Rbin",
    xaxis_title="Rbin",
    yaxis_title="\u0394R\u03C3"
)

In [197]:
plot_df_val = pd.DataFrame()
# validation data
plot_df_val['v'] = x_val['v']
plot_df_val['Rbin'] = x_val['Rbin']
plot_df_val['y_val'] = y_val
plot_df_val['y_val_pred'] = y_val_pred

plot_df_val = plot_df_val.sort_values('v')

In [198]:
colours = ['#636EFA', '#EF553B', '#00CC96', '#AB63FA', '#FFA15A', '#19D3F3', '#FF6692', '#B6E880', '#FF97FF', '#FECB52',
           '#636EFA', '#EF553B', '#00CC96', '#AB63FA', '#FFA15A', '#19D3F3', '#FF6692', '#B6E880', '#FF97FF', '#FECB52'
           ]
fig = go.Figure()

for val, col in zip(plot_df_val['v'].unique(), colours):
    data = plot_df_val.loc[plot_df_val['v']==val]
    fig.add_trace(go.Scatter(y = data['y_val'],
                             x = data['Rbin'],
                             marker=dict(color=col),
                             mode='markers',
                             name=f'{val}V'
                             ))
    fig.add_trace(go.Scatter(y = data['y_val_pred'],
                             x = data['Rbin'],
                             line=dict(color='red'),
                             mode='markers',
                             name=f'{val}V'
                             ))
fig.update_layout(
    title="\u0394R\u03C3 vs Rbin",
    xaxis_title="Rbin",
    yaxis_title="\u0394R\u03C3"
)

In [199]:
fig = go.Figure()

for val1, col1 in zip(plot_df_val['v'].unique(), colours):
    data1 = plot_df_val.loc[plot_df_val['v']==val1]
    # fig.add_trace(go.Scatter(y = data1['y_val'],
    #                          x = data1['Rbin'],
    #                          marker=dict(color='red'),
    #                          mode='markers',
    #                          name=f'{val1}V'
    #                          ))
    fig.add_trace(go.Scatter(y = data1['y_val_pred'],
                             x = data1['Rbin'],
                             marker=dict(color=col1),
                             mode='markers',
                             name=f'{val1}V'
                             ))
for val, col in zip(plot_df['v'].unique(), colours):
    data = plot_df.loc[plot_df['v']==val]
    # fig.add_trace(go.Scatter(y = data['Y_test'],
    #                          x = data['Rbin'],
    #                          marker=dict(color=col),
    #                          mode='markers',
    #                          name=f'{val}V'
    #                          ))
    fig.add_trace(go.Scatter(y = data['y_test_pred'],
                             x = data['Rbin'],
                             marker=dict(color='red'),
                             mode='markers',
                             name=f'{val}V'
                             ))
fig.update_layout(
    title="\u0394R\u03C3 vs Rbin",
    xaxis_title="Rbin",
    yaxis_title="\u0394R\u03C3"
)